<a href="https://colab.research.google.com/github/shrutishrinivasan/CIA_DAA/blob/main/AntColonyOptimization(NN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv("loan.csv")
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [ ]:
df = data
# removing irrelevant columns like Id, Zip code
# and shifting the target variable to the last position
df = df.drop(['ID','ZIP Code','Personal Loan'],axis=1)
# Target variable: Personal Loan
df['Perosnal Loan'] = data['Personal Loan']
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Perosnal Loan
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0


In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.25)
X_train

array([[ 51.,  25., 163., ...,   0.,   1.,   0.],
       [ 42.,  17.,  44., ...,   1.,   1.,   1.],
       [ 35.,  11.,  24., ...,   0.,   0.,   0.],
       ...,
       [ 26.,   1.,  24., ...,   0.,   0.,   1.],
       [ 57.,  33.,  49., ...,   1.,   1.,   1.],
       [ 33.,   9.,  38., ...,   0.,   0.,   0.]])

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

In [ ]:
# Define the neural network architecture
model = Sequential()
model.add(Dense(16, input_dim=11, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
# Define the objective function
model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
# Initialize the ACO algorithm
import numpy as np
num_ants = 11
num_iterations = 50
pheromone_trails = np.ones((num_ants, model.count_params()))
best_ant_weights = None
best_ant_score = 0

In [ ]:
# Define the crossover operator
def crossover(ant1, ant2):
    mask = np.random.randint(0, 2, ant1.shape).astype(bool)
    new_ant = np.zeros_like(ant1)
    new_ant[mask] = ant1[mask]
    new_ant[~mask] = ant2[~mask]
    return new_ant

In [ ]:
def mutate(ant, mutation_rate):
    mask = np.random.uniform(size=ant.shape) < mutation_rate
    new_ant = ant + mask * np.random.normal(scale=0.1, size=ant.shape)
    return new_ant

In [23]:
# Main ACO loop
for iteration in range(num_iterations):
    # Create a new colony of ants
    new_colony = []

    for ant_index in range(num_ants):
        # Generate a new ant
        ant_weights = np.zeros(model.count_params())

        for i in range(len(pheromone_trails[ant_index])):
            # Sample the weight using a softmax function
            ant_weights[i] = np.exp(pheromone_trails[ant_index][i])
        ant_weights = ant_weights / np.sum(ant_weights)

        # Evaluate the ant's performance on the training set
        model.fit(X_train, keras.utils.to_categorical(y_train, num_classes=2), epochs=10, verbose=0)
        score = model.evaluate(X_train, keras.utils.to_categorical(y_train, num_classes=2), verbose=0)[1]

        # Update the best ant if necessary
        if score > best_ant_score:
            best_ant_score = score
            best_ant_weights = ant_weights

        # Add the ant to the new colony
        new_colony.append(ant_weights)

    # Update the pheromone trails of the ants
    for ant_index in range(num_ants):
        pheromone_trails[ant_index] = pheromone_trails[ant_index] * (1 - 0.1)
        if np.array_equal(new_colony[ant_index], best_ant_weights):
            pheromone_trails[ant_index] += 0.1 / best_ant_score
        else:
            pheromone_trails[ant_index] += 0.1

    # apply mutation and crossover
    for ant_index in range(num_ants-1):
      crossover(ant_index,ant_index+1)

In [24]:
def implement_model(X_train,y_train,X_test,y_test):
  model = Sequential()
  model.fit(X_train,y_train)
  accuracy = model.evaluate(X_test,y_test)
  return accuracy

In [25]:
acc = implement_model(X_train,y_train,X_test,y_test)
acc

0.8952
